In [ ]:
import gzip
from monty.json import MontyDecoder
from ujson import load
from pymatgen.core.structure import Molecule, Structure
from pathlib import Path
from time import time
from mpcontribs.client import Client
from tqdm.auto import tqdm

In [ ]:
name = "open_catalyst_project"
client = Client()

In [ ]:
p = Path("/Users/patrick/is2res_train/is2res_train")
jsons = list(p.glob("*.json.gz"))

In [ ]:
decoder = MontyDecoder()

def get_contribution(path):
    
    if path.stat().st_size / 1024 / 1024 > 10:
        return None
    
    with gzip.open(path) as f:
        data = decoder.process_decoded(load(f))
    
    struct = data['trajectory'][-1]
    struct.add_site_property('tags', [int(t) for t in data['tags']])

    mol = Molecule.from_sites([site for site in struct if site.properties['tags'] == 2])
    iupac_formula = mol.composition.iupac_formula
    bulk_struct = Structure.from_sites([site for site in struct if site.properties['tags'] != 2])
    bulk_formula = bulk_struct.composition.reduced_formula

    search_data = {
        "mpid": data['bulk_id'],
        "adsorptionEnergy": data["adsorption_energy"],
        "adsorbateSmiles": data["adsorbate_smiles"],
        "adsorbateIUPACFormula": iupac_formula,
        "bulkFormula": bulk_formula,
        "h": data["surface_miller_indices"][0],
        "k": data["surface_miller_indices"][1],
        "l": data["surface_miller_indices"][2],
        "surfaceTop": data["surface_top"],
        "surfaceShift": data["surface_shift"]
    }

    return {
        "project": name,
        "formula": struct.composition.reduced_formula,
        "identifier": data["id"],
        "data": search_data,
        "structures": [struct],
        "attachments": [path]
    }

In [ ]:
client.get_totals({"project": name})

In [ ]:
all_ids = client.get_all_ids({"project": name}).get(name, {}).get("identifiers", set())
print(len(all_ids))

contributions, cnt = [], 0

for path in tqdm(jsons):
    if Path(path.stem).stem not in all_ids:
        contrib = get_contribution(path)
        if not contrib:
            continue
            
        contributions.append(contrib)
        cnt += 1
        
        if not cnt % 5:
            client.submit_contributions(
                contributions, per_request=10,
                ignore_dupes=True, skip_dupe_check=True, retry=True
            )
            contributions.clear()
            
print(cnt)